# Сохранение данных в Firebase

In [ ]:
# !pip install firebase_admin

In [50]:
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import pandas as pd
import json

In [51]:
### Весь процесс в 1 функции 

In [52]:
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import pandas as pd
import json

# Здесь типо парсинг
#Но для примера я тупо читаю с гитхаба
СonstitutionalReferendum = pd.read_csv("https://raw.githubusercontent.com/EmilStasevski/Parsing-CEC-Central-Election-Commission-of-the-Russian-Federation/main/data/republic_of_adygeya.csv")
PresidentElection = pd.read_csv("https://raw.githubusercontent.com/EmilStasevski/Parsing-CEC-Central-Election-Commission-of-the-Russian-Federation/main/data/df.csv")
StateDuma = pd.DataFrame()
RegionalElection = pd.DataFrame()
# Все 4 спарсенных датафрема - (Президентские,Региональные , Конституция,  ГосДума)
list_of_dfs = [PresidentElection, RegionalElection,СonstitutionalReferendum , StateDuma  ]

def pushFireBase(list_of_dfs):
    json_list = [i.to_dict(orient='records') for i in list_of_dfs]
    collections_names = [ 'PresidentElection', 'RegionalElection','СonstitutionalReferendum', 'StateDumaElection']
    cred = credentials.Certificate("CEC.json")
    if not firebase_admin._apps:
        firebase_admin.initialize_app(cred, {
        'databaseURL': "https://parsingcec-default-rtdb.europe-west1.firebasedatabase.app/"
    })
    ref = db.reference('/')
    for name, df in zip(collections_names, json_list):
        ref.child(name).set(df)
        
pushFireBase(list_of_dfs)

### Коннектимся с базой

In [45]:
cred = credentials.Certificate("CEC.json")
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': "https://parsingcec-default-rtdb.europe-west1.firebasedatabase.app/"
    })

In [46]:
ref = db.reference('/')

In [47]:
constRF= pd.read_csv("https://raw.githubusercontent.com/EmilStasevski/Parsing-CEC-Central-Election-Commission-of-the-Russian-Federation/main/data/republic_of_adygeya.csv")
president = pd.read_csv("https://raw.githubusercontent.com/EmilStasevski/Parsing-CEC-Central-Election-Commission-of-the-Russian-Federation/main/data/df.csv")
dfs = [constRF, president]

### Превращаем df в json

In [48]:
president_json = president.to_dict(orient='records')
constRF_json = constRF.to_dict(orient='records')
constRF_json[0]

{'region': 'Республика Адыгея (Адыгея)',
 'commission_name': 'Адыгейская',
 'voters_number': 11669,
 'issued_ballots_number': 9810,
 'turned_ballots_number': 9810,
 'bad_ballots_number': 30,
 'for_votes_number': 8637,
 'for_votes_percent': 88.03999999999999,
 'against_votes_number': 1143,
 'against_votes_percent': 11.65}

### Пушим json в базу данных

In [49]:
ref.set({
    # Президентские выборы
    "PresidentElection":
    president_json,
    
    # поправки в Конституцию
    "ConstRF":
    constRF_json 
    
})